### Command Pattern
Suppose we want to perform an action, as in example below:

In [ ]:
public class Client{
    public static void main(String[] args)    {
        RemoteControl control = new RemoteControl();
        
        Light light = new Light();
        
        // Switch on the light
        light.switchOn();
        
        // Switch off the light
        light.switchOff();
    }
}

Later a need may arise to
- log the actions performed
- store the actions to be performed and execute later
- provide support for undoing actions

In such scenario, we employ Command Pattern. The Command Pattern encapsulates a request as an object, thereby letting you parameterize other objects with different requests, queue or log requests, and support undoable operations. 

### Implementation
**Command Interface:**

In [ ]:
public interface Command{
    public void execute();
}

**Concrete implementation:**

In [ ]:
public class LightOnCommand implements Command{
    // Light is Receiver
    Light light;
    
    public LightOnCommand(Light light){
        this.light = light;
    }
    
    public void execute(){
        light.switchOn();
    }
}

public class LightOffCommand implements Command{
    // Light is Receiver
    Light light;
    
    public LightOffCommand(Light light){
        this.light = light;
    }
    
    public void execute(){
        light.switchOff();
    }
}

**Receiver:**

In [ ]:
public class Light{
    private boolean on;
    
    public void switchOn(){
        on = true;
    }
    
    public void switchOff(){
        on = false;
    }
}

**Invoker:** the purpose of Invoker is to act as a central place from where all the commands will be executed. It serves as command storage facility.

In [ ]:
public class RemoteControl{
    private Command command;
    
    public void setCommand(Command command){
        this.command = command;
    }
    
    public void pressButton(){
        command.execute();
    }
}

**Client Code:**

In [ ]:
public class Client{
    public static void main(String[] args)    {
        RemoteControl control = new RemoteControl();
        
        Light light = new Light();
        Command lightsOn = new LightsOnCommand(light);
        Command lightsOff = new LightsOffCommand(light);

        //Switch on
        control.setCommand(lightsOn);
        control.pressButton();

        //Switch off
        control.setCommand(lightsOff);
        control.pressButton();
    }
}

Instead of having reference to one single command, invoker can reference a list of commands. 
    
### Undo Functionality

In [ ]:
//Reciver
public class Fan {
    public enum Speed {
        LOW, MEDIUM, HIGH
    };

    private Speed speed;
    private boolean isOn;

    public Speed getSpeed() {
        return this.speed;
    }

    public void switchOn() {
        this.isOn = true;
    }

    public void switchOff() {
        this.isOn = false;
    }

    public void setSpeed(Speed speed) {
        this.speed = speed;
    }
}

// Command
public interface Command {
    public void execute();

    public void undo();
}

// Concrete command
public class FanHighSpeedCommand implements Command {
    private Fan fan;
    private Fan.Speed prevSpeed;

    public FanHighSpeedCommand(Fan fan) {
        this.fan = fan;
    }

    @Override
    public void execute() {
        this.prevSpeed = fan.getSpeed();
        this.fan.setSpeed(Fan.Speed.HIGH);
    }

    @Override
    public void undo() {
        if (prevSpeed == Fan.Speed.HIGH) {
            fan.setSpeed(Fan.Speed.HIGH);
        } else if (prevSpeed == Fan.Speed.MEDIUM) {
            fan.setSpeed(Fan.Speed.MEDIUM);
        } else if (prevSpeed == Fan.Speed.LOW) {
            fan.setSpeed(Fan.Speed.LOW);
        }
    }
}

//Invoker
public class RemoteControl {
    private Map<String, Command> fanCommands = new HashMap<>();

    private Stack<Command> undoStack = new Stack<Command>();

    public void setFanSpeedCommand(Command command, String state) {
        this.fanCommands.put(state, command);
    }

    public void fanSpeedButton(String speed) {
        if (speed.equals("HIGH")) {
            Command command = this.fanCommands.get("HIGH");
            command.execute();
            undoStack.add(command);
        }
    }

    public void fanStateButton(String state) {
        if (state.equals("ON")) {
            Command command = this.fanCommands.get("ON");
            command.execute();
            undoStack.add(command);
        } else if (state.equals("OFF")) {
            Command command = this.fanCommands.get("OFF");
            command.execute();
            undoStack.add(command);
        }
    }

    public void undoButton() {
        Command command = undoStack.pop();
        command.undo();
    }
}